# Here we go again...

In [7]:
from simpletransformers.language_representation import RepresentationModel
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random

In [8]:
data = pd.read_json('data/additional/scraped_data_raw.json')
data.head()

,index,Id,Domain,Body,Header,PublicationDate,Uri,Byline,TextHash
0,0,842325,politiken.dk,Læs artiklen senere Gemt (klik for at fjerne) ...,Utilfreds passager: »Prøv selv en tur klokken ...,2002-09-17T00:00:00,https://politiken.dk/forbrugogliv/art6088327/U...,Annemette Grundtvig,-346867430
1,1,842324,politiken.dk,Klimamonitor Byrummonitor Skolemonitor Sundhed...,Minimalist: »Ofte står jeg og mangler en ting ...,2013-01-18T00:00:00,https://politiken.dk/forbrugogliv/art6287464/%...,Annemette Grundtvig,-1346564151
2,2,842323,politiken.dk,Læs artiklen senere Gemt (klik for at fjerne) ...,Digitalt forældreskab: Hvilken type er du som ...,2006-05-17T00:00:00,https://politiken.dk/forbrugogliv/art5934818/H...,Annemette Grundtvig,1473032676
3,3,842322,politiken.dk,FAKTA: Kemi i mademballage af papir Forbrugerr...,Ny test afslører: Denne kaffekop og dette suge...,2004-07-19T00:00:00,https://politiken.dk/forbrugogliv/art7278657/D...,Annemette Grundtvig,1097982200
4,4,842321,politiken.dk,Læs artiklen senere Gemt (klik for at fjerne) ...,"Pakkeshop, pick-up eller ekspres? Pakkernes sv...",2010-12-19T00:00:00,https://politiken.dk/forbrugogliv/art7524916/P...,Annemette Grundtvig,-2079936539


In [ ]:
politiken = list(data[data.Domain == 'politiken.dk'].Header)[:1000]
information = list(data[data.Domain == 'information.dk'].Header)[:1000]
len(politiken), len(information)

In [ ]:
%%time
comb = politiken+information
model = RepresentationModel(
        model_type='bert',
        model_name='Maltehb/danish-bert-botxo',
        use_cuda=False)

word_vectors = model.encode_sentences(comb, combine_strategy='mean')

In [ ]:
word_vectors.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
y = [0]*1000 + [1]*1000
train_X, test_X, train_y, test_y = train_test_split(word_vectors, y)

In [ ]:
rfc = RandomForestClassifier().fit(train_X, train_y)

In [ ]:
rfc.score(test_X, test_y)

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import *

dum = DummyClassifier().fit(train_X, train_y)
dum.score(test_X, test_y)

# Test the shit on codified

In [ ]:
codified = np.load('../../../codified.npy')
codified.shape

In [ ]:
text = list(data.Header.astype(str))
texts = text
final_text = ''
for text in tqdm(texts):
    text = text.strip()
    text = text.replace('\n','')
    text = text.replace('\r','')
    text = text.replace('\t','')
    if len(text) < 10:
        continue
    final_text += ''.join(text)
    final_text += '\n'
final_text = final_text[:-1]

headers = final_text.split('\n')

In [ ]:
politiken2_ind = []
information2_ind = []
politiken2 = []
information2 = []

for i, header in tqdm(enumerate(headers)):
    if header in politiken:
        politiken2_ind.append(i)
    elif header in information:
        information2_ind.append(i)
    
    if len(politiken2) == 1000 and len(information2)==1000:
        break

In [ ]:
politiken2_vecs = np.array([codified[i] for i in politiken2_ind])
information2_vecs = np.array([codified[i] for i in information2_ind][:992])
codified_vecs = np.vstack((politiken2_vecs, information2_vecs))

y = [0]*992 + [1]*992
train_X, test_X, train_y, test_y = train_test_split(codified_vecs, y)

In [ ]:
# random.shuffle(y)
# train_X, test_X, train_y, test_y = train_test_split(codified_vecs, y)

In [ ]:
import classifier_unit_test

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
classifier_unit_test.test_classifier(rfc, train_X, test_X, train_y, test_y)

In [ ]:
auc

# Generation test

In [37]:
#Make vector of some articles
bodies = list(data["Body"])[9000:9100]
headers = list(data["Header"])[9000:9100]

model = RepresentationModel(
        model_type='bert',
        model_name='Maltehb/danish-bert-botxo',
        use_cuda=False)

vectors = model.encode_sentences(bodies, combine_strategy='mean')
vectors.shape

Some weights of the model checkpoint at Maltehb/danish-bert-botxo were not used when initializing BertForTextRepresentation: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(100, 768)

In [30]:
train_data = [[vectors[i], headers[i]] for i in range(len(headers)-10)]
eval_data = [[vectors[i], headers[i]] for i in range(len(headers)-10, len(headers))]
train_df = pd.DataFrame(train_data, columns=["input", "target"])
eval_df = pd.DataFrame(eval_data, columns=["input", "target"])
train_df.head()

,input,target
0,"[-0.59563667, 0.46274945, 0.83595693, 0.591673...",Lettet Christinna Pedersen: 'Der har været for...
1,"[-0.3861792, 0.122922584, 0.9676417, -0.243358...",Ny museumsdirektør på kvindemuseet i Aarhus si...
2,"[-0.6130347, 0.15502118, 0.515855, 0.4008748, ...",Kulturministeren garanterer fans på stadion: '...
3,"[-0.38609523, 0.20278929, 0.98600316, 0.281827...",Debutanten Dolberg: Jeg kunne høre folk råbe p...
4,"[-0.7306787, 0.25296178, 1.0223695, 0.42101982...",Genoplev Danmarks vej til VM-finalen: Knusende...


In [39]:
from transformers import MarianMTModel, MarianTokenizer
import torch

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
embeddings = model.get_input_embeddings()

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]